# A Crash-Course on Pandas

A 30-minute intro to the main functionality of the `pandas` package. Please also see the [official pandas tutorials](https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html) on which this notebook is heavily based. I also recommend the ["10 minutes to pandas"](https://pandas.pydata.org/docs/user_guide/10min.html) crash-course on the [offical docs website](https://pandas.pydata.org/docs/index.html). 

In [1]:
import pandas as pd
import numpy as np

Pandas is a package for manipulating data. I like to think of pandas as Python's version of a spreadsheet. There are two main types of data structures in pandas, Series (1-D; a single column), and DataFrame (2-D; a table), which is similar to (but more powerful than) a data.frame in R. DataFrames are capable of everything a spreadsheet is, and are also capable of handling relational data similar to SQL-based relational databases.

Let's look at both in turn:

## Series

A Series is a labeled 1-D dataset. They can be instantiated manually, or extracted from a DataFrame. To build one manually,

In [2]:
ex_series = pd.Series([87, 91, 98, 85, 90], name="test scores")
print(ex_series)

0    87
1    91
2    98
3    85
4    90
Name: test scores, dtype: int64


Note that the data is automatically labeled with an integer index. We can specify our own index, say a 4-digit student ID:

In [3]:
test_scores = pd.Series(
	[87, 91, 98, 85, 90], 
	name="test scores", 
	index=["0001", "0002", "0003", "0004", "0005"]
)
print(test_scores)

0001    87
0002    91
0003    98
0004    85
0005    90
Name: test scores, dtype: int64


Data are accessed using their index, NOT their position in the series. Note that you *can* technically access custom-indexed data by its position, but pandas will give you a warning. The correct way to access data by position is to use the iloc method:

In [4]:
print(ex_series[1])
print(test_scores["0003"])
print(test_scores.iloc[3])

91
98
85


Series can also be turned into numpy arrays:

In [5]:
scores_array = test_scores.to_numpy()
print(scores_array)
print(type(scores_array))

[87 91 98 85 90]
<class 'numpy.ndarray'>


## Data Frames

A DataFrame is a 2-D labeled dataset organized into a table with rows and columns. We can also make these manually using dictionaries:

In [6]:
data_dict = {
	"Student ID": ["0001", "0002", "0003", "0004", "0005"],
	"Age": [13, 14, 14, 13, 13],
	"Score": [87, 91, 98, 85, 90]
}
ex_df = pd.DataFrame(data_dict)
print(ex_df)

  Student ID  Age  Score
0       0001   13     87
1       0002   14     91
2       0003   14     98
3       0004   13     85
4       0005   13     90


To select a column, use its name like accessing a dict. Note that columns of a DataFrame are Series.

In [7]:
print(ex_df["Age"])
print(type(ex_df["Age"]))

# you can also select more than one column with a list:
print(ex_df[["Student ID","Age"]])

0    13
1    14
2    14
3    13
4    13
Name: Age, dtype: int64
<class 'pandas.core.series.Series'>
  Student ID  Age
0       0001   13
1       0002   14
2       0003   14
3       0004   13
4       0005   13


To select (a) row(s), you have a few options. I think the easiest is to use a boolean array. This is the operation that you're wrapping in ParaFrame's filter.

In [8]:
print(ex_df[ex_df["Student ID"] == "0001"])
print(ex_df[ex_df["Score"] >= 90])

  Student ID  Age  Score
0       0001   13     87
  Student ID  Age  Score
1       0002   14     91
2       0003   14     98
4       0005   13     90


Like Series, you can also use the loc and iloc methods to select rows by index and position, respectively:

In [27]:
# let's add a manual index
ex_df.index = ["Student 1", "Student 2", "Student 3", "Student 4", "Student 5"]

print(ex_df.loc["Student 5"])
print(ex_df.iloc[0])

# Note these return Series as well!
print(type(ex_df.iloc[0]))

Student ID    0005
Age             13
Score           90
Name: Student 5, dtype: object
Student ID    0001
Age             13
Score           87
Name: Student 1, dtype: object
<class 'pandas.core.series.Series'>


## I/O

In addition to manual creation, pandas can read and write dataframes to/from a variety of common file types, including csv, excel, hdf5, json, sql, and more! These operations are performed with the `DataFrame.read_*` and `DataFrame.to_*` methods. For more info, see the [docs](https://pandas.pydata.org/docs/user_guide/io.html). For example, to read a csv file, do the following:

In [10]:
# let's grab the Titanic data file that pandas offers for its tutorials. 
import requests
import io

data_url = "https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv"
response = requests.get(data_url).content
# Note that if you already have a file on-hand, you just pass the file location to read_csv()
df = pd.read_csv(io.StringIO(response.decode('utf-8')))

print(df)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                                Heikkinen, Miss Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

Pandas auto-detects the data types of each column, but you can also specify them yourself with the `dtypes` kwarg.

In [11]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

## Cleaning data
As we saw in Evan's last debugging example, missing or bad data propagates through calculations using dataframes. To deal with bad or missing data, pandas offers the dropna() method to drop rows with any missing data, and the fillna() method to replace missing data with a placeholder. For more info, see the [missing data guide](https://pandas.pydata.org/docs/user_guide/missing_data.html).

In [12]:
print(df.shape)
print(df.dropna().shape)
print(df.fillna(0).shape)

(891, 12)
(183, 12)
(891, 12)


## Creating new and derived columns
Creating new columns is as easy as assigning an array or series of the proper dimension to an unused column name. This can also be used to create columns derived from existing ones. 

In [13]:
# add a column manually
df["Brought Cellphone"] = np.zeros(df.shape[0], dtype=int)
print(df)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                                Heikkinen, Miss Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [14]:
# add a derived column
df["Fare Tax"] = 0.08 * df["Fare"]
print(df)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                                Heikkinen, Miss Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

## Summary Statistics
Pandas has some built-in functionality for calulating summary stats for columns. Of course, it's always possible to extract a column to an array (as shown previously) and perform whataver fancy operations you want. 

In [15]:
# eveything at once
df["Age"].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [16]:
# or just one at a time
print(df["Age"].mean())
print(df["Age"].std())

29.69911764705882
14.526497332334042


## Joining, Merging, and Concatenation
Pandas also offers a few different methods for combining multiple tables. I think the [documentation](https://pandas.pydata.org/docs/user_guide/merging.html) has some good examples, so I'm replicating some of these here.

In [ ]:
# Concatenation joins two data frames along the specified axis, with the default being column-wise.
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)

df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
)

# You can specify the axis to operate on, the default is 0.
print(pd.concat([df1, df2], axis=0))
print(pd.concat([df1, df2], axis=1))

    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1
2  A2  B2  C2  D2
3  A3  B3  C3  D3
4  A4  B4  C4  D4
5  A5  B5  C5  D5
6  A6  B6  C6  D6
7  A7  B7  C7  D7
     A    B    C    D    A    B    C    D
0   A0   B0   C0   D0  NaN  NaN  NaN  NaN
1   A1   B1   C1   D1  NaN  NaN  NaN  NaN
2   A2   B2   C2   D2  NaN  NaN  NaN  NaN
3   A3   B3   C3   D3  NaN  NaN  NaN  NaN
4  NaN  NaN  NaN  NaN   A4   B4   C4   D4
5  NaN  NaN  NaN  NaN   A5   B5   C5   D5
6  NaN  NaN  NaN  NaN   A6   B6   C6   D6
7  NaN  NaN  NaN  NaN   A7   B7   C7   D7


In [26]:
# if the two dataframes have different columns, you should specify an "inner" or "outer" join:
df3 = pd.DataFrame(
    {
        "B": ["B2", "B3", "B4", "B5"],
        "C": ["C2", "C3", "C4", "C5"],
        "D": ["D2", "D3", "D4", "D5"],
        "E": ["E2", "E3", "E4", "E5"],
    },
    index=[2, 3, 4, 5],
)

# outer takes the union along the specified axis
print(pd.concat([df1, df3], join="outer", axis=1))

# inner takes the overlap along the specified axis
print(pd.concat([df1, df3], join="inner", axis=1))

     A    B    C    D    B    C    D    E
0   A0   B0   C0   D0  NaN  NaN  NaN  NaN
1   A1   B1   C1   D1  NaN  NaN  NaN  NaN
2   A2   B2   C2   D2   B2   C2   D2   E2
3   A3   B3   C3   D3   B3   C3   D3   E3
4  NaN  NaN  NaN  NaN   B4   C4   D4   E4
5  NaN  NaN  NaN  NaN   B5   C5   D5   E5
    A   B   C   D   B   C   D   E
2  A2  B2  C2  D2  B2  C2  D2  E2
3  A3  B3  C3  D3  B3  C3  D3  E3


In [ ]:
# Merging is for SQL-type joining operations (joining based on a key/column value). Again taking the example from the docs,
left = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

right = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

# when merging, specify which column to join on:
print(pd.merge(left, right, on="key"))

  key   A   B   C   D
0  K0  A0  B0  C0  D0
1  K1  A1  B1  C1  D1
2  K2  A2  B2  C2  D2
3  K3  A3  B3  C3  D3


In [ ]:
# You can specify the merge type with the "how" kwarg:
left = pd.DataFrame(
   {
      "key1": ["K0", "K0", "K1", "K2"],
      "key2": ["K0", "K1", "K0", "K1"],
      "A": ["A0", "A1", "A2", "A3"],
      "B": ["B0", "B1", "B2", "B3"],
   }
)

right = pd.DataFrame(
   {
      "key1": ["K0", "K1", "K1", "K2"],
      "key2": ["K0", "K0", "K0", "K0"],
      "C": ["C0", "C1", "C2", "C3"],
      "D": ["D0", "D1", "D2", "D3"],
   }
)

print(pd.merge(left, right, on=["key1", "key2"], how="left"))
print(pd.merge(left, right, on=["key1", "key2"], how="right"))
print(pd.merge(left, right, on=["key1", "key2"], how="outer"))
print(pd.merge(left, right, on=["key1", "key2"], how="inner"))

  key1 key2   A   B    C    D
0   K0   K0  A0  B0   C0   D0
1   K0   K1  A1  B1  NaN  NaN
2   K1   K0  A2  B2   C1   D1
3   K1   K0  A2  B2   C2   D2
4   K2   K1  A3  B3  NaN  NaN
  key1 key2    A    B   C   D
0   K0   K0   A0   B0  C0  D0
1   K1   K0   A2   B2  C1  D1
2   K1   K0   A2   B2  C2  D2
3   K2   K0  NaN  NaN  C3  D3
  key1 key2    A    B    C    D
0   K0   K0   A0   B0   C0   D0
1   K0   K1   A1   B1  NaN  NaN
2   K1   K0   A2   B2   C1   D1
3   K1   K0   A2   B2   C2   D2
4   K2   K0  NaN  NaN   C3   D3
5   K2   K1   A3   B3  NaN  NaN
  key1 key2   A   B   C   D
0   K0   K0  A0  B0  C0  D0
1   K1   K0  A2  B2  C1  D1
2   K1   K0  A2  B2  C2  D2


In [48]:
# join is a DataFrame method and is used to join two dataframes based on columns:
left = pd.DataFrame(
    {
		"A": ["A0", "A1", "A2"], 
		"B": ["B0", "B1", "B2"]
	}
)

right = pd.DataFrame(
    {	
		"C": ["C0", "C2", "C3"], 
		"D": ["D0", "D2", "D3"]
	}, 
	index=["K0", "K2", "K3"]
)

print(left.join(right))
print(right.join(left))

    A   B    C    D
0  A0  B0  NaN  NaN
1  A1  B1  NaN  NaN
2  A2  B2  NaN  NaN
     C   D    A    B
K0  C0  D0  NaN  NaN
K2  C2  D2  NaN  NaN
K3  C3  D3  NaN  NaN


## Documentation

As usual, I've barely scratched the surface of what pandas is capable of. If you want to learn more, there's an excellent [user guide](https://pandas.pydata.org/docs/user_guide/index.html) on the pandas website. 